<a href="https://colab.research.google.com/github/Heity94/TWSM_Lab/blob/main/Project/notebooks/PH_NewEntityRecognition_VanillaST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [103]:
# Declare whether you are on Colab or local
colab = True

In [104]:
if colab==True:
  
  #Mount drive
  from google.colab import drive
  drive.mount('/content/drive')

  #set path to data in Google Drive
  data_path = "/content/drive/MyDrive/2022_Analytics Lab Student Projects/Data/All Topics"
  data_path_group = data_path[:-10]+"Topic 1/Data_Team1/" # create new data path to access files created by Team1

  #install required packages
  #!pip install duckdb
  #!pip install -U sentence-transformers -q


else:
    data_path_group = "../data/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [105]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Enhance ontology to also include parent entity id

In [106]:
#Load df of entities which were removed from the ontology during the train-val-est split
ent_removed_df = pd.read_csv(data_path_group+"NER/Data_to_split_ontology/NER_ent_to_remove_from_ontology_final.csv", index_col=0)

In [107]:
ent_removed_df.head()

,level_1,level_2,entity_id_to_remove,has_child_nodes,lvl_ent,lvl_par_ent,par_ent,ent_list,articles_list,sentences_list,num_ent,num_articles,num_sentences
0,domain specific entity,IS topic,business process outsourcing,1.0,5,4,outsourcing,"['business process outsourcing', 'knowledge pr...","[5414, 5218, 440, 5609, 5610, 5611, 1195, 2659...","['5414_1618_1639', '5218_4403_4488', '440_8401...",3,130,351
1,domain specific entity,IS topic,cybersecurity behavior,1.0,5,4,IT security category,"['privacy behavior', 'information security pol...","[1622, 1018, 3375, 2474, 6330, 5947, 1845, 425...","['1622_3154_3221', '1622_3221_3320', '1622_334...",4,121,695
2,domain specific entity,IS topic,user interface,1.0,4,3,human–computer interaction,"['user interface', 'graphical user interface',...","[1836, 434, 4244, 3373, 5413, 5415, 3196, 5606...","['1836_5404_5485', '434_1460_1510', '4244_8878...",5,980,2694
3,domain specific entity,IS topic,IT planning,1.0,4,3,information systems strategy,"['IT planning', 'IT policy', 'IT strategic pla...","[1834, 4246, 3025, 2467, 4631, 2848, 2851, 369...","['1834_10730_10748', '1834_10941_10959', '4246...",4,728,3431
4,domain specific entity,IS topic,business process management,1.0,3,2,IS topic,"['business process improvement', 'business pro...","[190, 1835, 1836, 432, 4244, 4245, 4246, 3372,...","['190_5779_5825', '190_6002_6027', '190_6702_6...",10,2148,13720


In order to determine how well our model is finding new entities within the sentences/noun phrases, we have to compare the closest entity which our model finds with the "true" parent entity id. For that we need to merge this informatin on the validation and test set. 

In [108]:
#load validation set 
ontology_val = pd.read_csv(data_path_group+"NER/NER_ontology_val.csv", index_col=0)

In [150]:
ontology_val

,entity_id,category,label,synonym
633,business process outsourcing,domain specific entity,TOPIC,outsourc business process
634,business process outsourcing,domain specific entity,TOPIC,outsourc business processes
635,business process outsourcing,domain specific entity,TOPIC,outsourced business processes
636,business process outsourcing,domain specific entity,TOPIC,outsourced business process
637,business process outsourcing,domain specific entity,TOPIC,business process outsourcings
...,...,...,...,...
29,predictive validity,methodological entity,VALIDITY,predictive validities
30,predictive validity,methodological entity,VALIDITY,predictive criterion validity
31,predictive validity,methodological entity,VALIDITY,predictive criterion validities
52,formative validity,methodological entity,VALIDITY,formative validities


In [153]:
# merge entities which were removed from train on ontology (inner merge, because ent_removed_df also includes entities which have been for the test set)
ontology_val_df = ontology_val.merge(ent_removed_df, how="inner", left_on="entity_id", right_on="entity_id_to_remove").drop(columns=["category", "label", "entity_id_to_remove"])

In [155]:
ontology_val_df.head(3)

,entity_id,synonym,level_1,level_2,has_child_nodes,lvl_ent,lvl_par_ent,par_ent,ent_list,articles_list,sentences_list,num_ent,num_articles,num_sentences
0,business process outsourcing,outsourc business process,domain specific entity,IS topic,1.0,5,4,outsourcing,"['business process outsourcing', 'knowledge pr...","[5414, 5218, 440, 5609, 5610, 5611, 1195, 2659...","['5414_1618_1639', '5218_4403_4488', '440_8401...",3,130,351
1,business process outsourcing,outsourc business processes,domain specific entity,IS topic,1.0,5,4,outsourcing,"['business process outsourcing', 'knowledge pr...","[5414, 5218, 440, 5609, 5610, 5611, 1195, 2659...","['5414_1618_1639', '5218_4403_4488', '440_8401...",3,130,351
2,business process outsourcing,outsourced business processes,domain specific entity,IS topic,1.0,5,4,outsourcing,"['business process outsourcing', 'knowledge pr...","[5414, 5218, 440, 5609, 5610, 5611, 1195, 2659...","['5414_1618_1639', '5218_4403_4488', '440_8401...",3,130,351


In [110]:
#load nounphrases val 
noun_phrases_val = pd.read_csv(data_path_group+"NER/NER_entities_val.csv", index_col=0)

Let's quickly check whether the sentences within the `noun_phrases_val` dataset are identical to the ones from the `ontology_val_df`

In [111]:
#unique number of sentences in ontology_val_df
from ast import literal_eval

# Change type from list stored as string to list
ontology_val_df["sentences_list"] = ontology_val_df.sentences_list.apply(lambda x: literal_eval(x))

In [112]:
# Create one big list of sentences
sentences_list = ontology_val_df.drop_duplicates("entity_id").sentences_list.sum(axis=0)

In [113]:
#remove duplicates
sentences_list_cln = list(set(sentences_list))
#check number of sentences
len(sentences_list_cln)

55300

In [114]:
# Compare against the number of unique sentences in the noun_phrases df
len(sentences_list_cln) == noun_phrases_val.sentence_id.unique().shape[0]

True

Perfect. Then we can continue. 

### Enhance noun phrases to also include "true" parent entity id 

In [156]:
cols = ["entity_id", "par_ent", "lvl_par_ent", "lvl_ent", "level_1", "level_2"]
noun_phrases_val_df = noun_phrases_val.merge(ontology_val_df[cols], how="left", left_on="true_ent_id", right_on="entity_id").drop(columns="entity_id").drop_duplicates()

In [157]:
noun_phrases_val_df

,article_id,sentence_id,ent_id,noun_phrases,true_ent_id,par_ent,lvl_par_ent,lvl_ent,level_1,level_2
0,189,189_4267_4292,causal link,We,0,NaN,NaN,NaN,NaN,NaN
1,189,189_4267_4292,causal link,this perverse nature,0,NaN,NaN,NaN,NaN,NaN
2,189,189_4267_4292,causal link,consequence,0,NaN,NaN,NaN,NaN,NaN
3,189,189_4267_4292,causal link,the notion,0,NaN,NaN,NaN,NaN,NaN
4,189,189_4267_4292,causal link,a method,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
653749,4993,4993_9181_9208,theoretical link,the post-adoption consequences,0,NaN,NaN,NaN,NaN,NaN
653750,4993,4993_9181_9208,theoretical link,IOS,0,NaN,NaN,NaN,NaN,NaN
653751,4993,4993_9181_9208,theoretical link,the relatively static manner,0,NaN,NaN,NaN,NaN,NaN
653752,4993,4993_9181_9208,theoretical link,which,0,NaN,NaN,NaN,NaN,NaN


Now our `noun_phrases_val_df` also includes the information about the "parent entity" of the noun phrases. We need this information to then compare the prediction from our model against the true parent entity, because ideally the model should give the parent entity id as the closest match. 

In [158]:
# Lets filter only noun phrases which have a true parent ent_id for now
noun_phrases_val_parents = noun_phrases_val_df[~noun_phrases_val_df.par_ent.isna()].copy()

In [159]:
noun_phrases_val_parents = noun_phrases_val_parents.drop_duplicates(subset="noun_phrases")
noun_phrases_val_parents.shape

(147, 10)

### Embedd noun phrases and ontology and find most similar entity per noun phrase

Now we will load the ontology_train, because this ontology does not include the entities or synonyms from the removed branches. 


In order to assess how well our model finds the correct parent entity id we will embedd the ontology train and the noun phrases, compute the cosine similarity and compare the result from prediction of our model (the closest entity to the noun phrase) and the true parent entity id. 

In [160]:
#load train set 
ontology_train = pd.read_csv(data_path_group+"NER/NER_ontology_train.csv", index_col=0)

In [161]:
# Import sentence_transformers 
#from sentence_transformers import SentenceTransformer, util
#from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, BinaryClassificationEvaluator, TripletEvaluator, LabelAccuracyEvaluator

In [162]:
# Load a pre-trained model
#model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [163]:
# create function based on semantic_search
def sem_search_sent_ent(model, sentences, entities, top_k = 1):


  #Compute embeddings
  embeddings_sentences = model.encode(sentences.noun_phrases.to_list(), convert_to_tensor=True, show_progress_bar=True)
  embeddings_entities = model.encode(entities.synonym.to_list(), convert_to_tensor=True, show_progress_bar=True)

  #Compute cosine-similarities for each sentence with all entities and return top k per sentence (list of list of dictonaries)
  cosine_scores = util.semantic_search(embeddings_sentences, embeddings_entities, top_k=top_k) #returns [[{'corpus_id': 483, 'score': 0.483295202255249}],...]

  #loop over results from semantic_search and create dataframe with sentence id and store in list
  dfs = []
  for idx, sim in enumerate(cosine_scores):
    test = pd.DataFrame(sim)
    test["sentence_id"] = sentences.iloc[idx,1]
    test["noun_phrase_txt"] = sentences.iloc[idx,3]
    test = test.rename(columns={"corpus_id":"entity_id"})
    dfs.append(test)

  # concat all dfs 
  dfs_df = pd.concat(dfs)
  dfs_df = dfs_df[["score", "sentence_id", "entity_id", "noun_phrase_txt"]] # reorder columns
  dfs_df[["main_entity_id"]] = dfs_df[["entity_id"]].applymap(lambda x:  entities.iloc[x,0]) # add "main" entity name (since we compare also with synonyms)
  dfs_df[["entity_id"]] = dfs_df[["entity_id"]].applymap(lambda x:  entities.iloc[x,3]) #replace entity_id_idx with real name of entity
  dfs_df = dfs_df.sort_values(by="score", ascending=False, ignore_index=True) # sort by score and reset index

  return dfs_df#, cosine_scores

In [164]:
#%%time
#sim_scores = sem_search_sent_ent(model, noun_phrases_val_parents, ontology_train, top_k=1)
#sim_scores.to_csv(data_path_group+"NER/similarities_validation.csv")

In [170]:
sim_scores = pd.read_csv(data_path_group+"NER/similarities_validation.csv", index_col=0)

In [171]:
sim_scores

,score,sentence_id,entity_id,noun_phrase_txt,main_entity_id
0,1.000000,5218_12213_12258,startups,startups,organizational participant
1,1.000000,6507_8810_8923,χ 2,χ 2,chi squared test
2,1.000000,5382_205_226,IS staffing,IS staffing,IT workforce
3,1.000000,5220_7434_7449,startup,startup,organizational participant
4,1.000000,3720_4861_4886,IT staffing,IT staffing,IT workforce
...,...,...,...,...,...
142,0.517927,1302_10308_10346,search engines,recommender engines,search engine
143,0.480481,6511_4459_4494,predictive analytics,recommender engine,machine learning
144,0.448159,1842_5705_5773,NLTK,typologies,NLTK
145,0.447968,6073_14795_14830,MIS implementation,confusion matrix,systems implementation


### Evaluate prediction results vs. true parent entity 

Ok now lets merge on these results the information regarding the "true" parent entity id

In [172]:
sim_scores_final = sim_scores.merge(
    noun_phrases_val_parents[['sentence_id', 'noun_phrases', 'par_ent', 'lvl_par_ent', 'lvl_ent', 'level_1', 'level_2']],
    how="left", left_on=["sentence_id", "noun_phrase_txt"], right_on=["sentence_id", "noun_phrases"]).drop(columns=["noun_phrases"])

In [173]:
sim_scores_final

,score,sentence_id,entity_id,noun_phrase_txt,main_entity_id,par_ent,lvl_par_ent,lvl_ent,level_1,level_2
0,1.000000,5218_12213_12258,startups,startups,organizational participant,company type,3.0,4.0,domain specific entity,study object
1,1.000000,6507_8810_8923,χ 2,χ 2,chi squared test,statistical metric,2.0,3.0,methodological entity,statistical metric
2,1.000000,5382_205_226,IS staffing,IS staffing,IT workforce,IT workforce,3.0,4.0,domain specific entity,IS topic
3,1.000000,5220_7434_7449,startup,startup,organizational participant,company type,3.0,4.0,domain specific entity,study object
4,1.000000,3720_4861_4886,IT staffing,IT staffing,IT workforce,IT workforce,3.0,4.0,domain specific entity,IS topic
...,...,...,...,...,...,...,...,...,...,...
142,0.517927,1302_10308_10346,search engines,recommender engines,search engine,analytical information system,4.0,5.0,domain specific entity,IS technology
143,0.480481,6511_4459_4494,predictive analytics,recommender engine,machine learning,analytical information system,4.0,5.0,domain specific entity,IS technology
144,0.448159,1842_5705_5773,NLTK,typologies,NLTK,model,2.0,3.0,theoretical entity,model
145,0.447968,6073_14795_14830,MIS implementation,confusion matrix,systems implementation,statistical metric,2.0,3.0,methodological entity,statistical metric


In [174]:
# compare prediction from model with true parent entity id
sim_scores_final["correct_match_parent"] = sim_scores_final.main_entity_id==sim_scores_final.par_ent

In [175]:
sim_scores_final.head(3)

,score,sentence_id,entity_id,noun_phrase_txt,main_entity_id,par_ent,lvl_par_ent,lvl_ent,level_1,level_2,correct_match_parent
0,1.0,5218_12213_12258,startups,startups,organizational participant,company type,3.0,4.0,domain specific entity,study object,False
1,1.0,6507_8810_8923,χ 2,χ 2,chi squared test,statistical metric,2.0,3.0,methodological entity,statistical metric,False
2,1.0,5382_205_226,IS staffing,IS staffing,IT workforce,IT workforce,3.0,4.0,domain specific entity,IS topic,True


In [221]:
#sim_scores_final.to_csv(data_path_group+"NER/similarities_validation_final.csv")

#### Overall accuracy

Now we can measure how well the out of the shelf sentence transformer mapped the noun phrase to the correct parent entity id

In [176]:
#overall accuracy 
np.round(sim_scores_final.correct_match_parent.sum()/sim_scores_final.correct_match_parent.count(),2)

0.24

#### Accuracy within similarity bins

We can also check in which ranges of cosine similarity the algorithm found the most matches

In [177]:
# Create bins of similarity score to check the number of correct matches compared to the labels of the database
grouped_df = sim_scores_final.groupby(pd.cut(sim_scores_final.score, np.arange(0.,1.1,0.1)))[["correct_match_parent"]]
corr_match_bins = np.round(grouped_df.sum()/grouped_df.count(),2).rename(columns={"correct_match_parent":"pct_correct_in_bin"})
corr_match_bins["no_correct"]= grouped_df.sum()
corr_match_bins["total"]= grouped_df.count()
corr_match_bins = corr_match_bins[["total", "no_correct", "pct_correct_in_bin"]]
corr_match_bins = corr_match_bins.iloc[::-1] #reverse order

In [178]:
corr_match_bins

,total,no_correct,pct_correct_in_bin
score,,,
"(0.9, 1.0]",8,1,0.12
"(0.8, 0.9]",42,18,0.43
"(0.7, 0.8]",28,11,0.39
"(0.6, 0.7]",39,5,0.13
"(0.5, 0.6]",23,0,0.00
"(0.4, 0.5]",4,0,0.00
"(0.3, 0.4]",0,0,NaN
"(0.2, 0.3]",0,0,NaN
"(0.1, 0.2]",0,0,NaN


#### Accuracy within the different levels/ domains of the ontology

We can also check if in some levels and categories the algorithm was performing better than in others

In [196]:
# Group matching results per level/domain and count number of correct matches and total noun phrases
matches_within_ontology_grouped = sim_scores_final.groupby(["level_1", "level_2", "lvl_ent"])\
  .aggregate(
    {'correct_match_parent' :["sum"], "noun_phrase_txt":["count"]})\
  .rename(columns={"noun_phrase_txt":"no_noun_phrases"})

In [217]:
# Calculate accuracy across each level/ domain
matches_within_ontology_grouped[("correct_match_parent", "pct")] = np.round(
    matches_within_ontology_grouped[('correct_match_parent','sum')]/matches_within_ontology_grouped[('no_noun_phrases', 'count')]
    ,2)

In [218]:
matches_within_ontology_grouped.head(3)

correct_match_parent  \
                                                                sum   
level_1                level_2         lvl_ent                        
domain specific entity Economic sector 3.0                        0   
                                       4.0                        0   
                                       5.0                        1   

                                               no_noun_phrases  \
                                                         count   
level_1                level_2         lvl_ent                   
domain specific entity Economic sector 3.0                   4   
                                       4.0                   2   
                                       5.0                   1   

                                               correct_match_parent  
                                                                pct  
level_1                level_2         lvl_ent                       
domain specific entity Economic sector 3.0                      0.0  
                                       4.0                      0.0  
                                       5.0                      1.0

To further enhance this grouped df we will also include the information on how many entities we originally remved from within this area of the ontology

In [205]:
# Group number of removed number and entites per level/domain 
rem_ent_ontology_grouped = ontology_val_df.drop_duplicates(subset="entity_id")\
    .groupby(["level_1", "level_2", "lvl_ent"])\
    .aggregate({'num_ent' :["sum","count"]})

In [219]:
rem_ent_ontology_grouped.head(3)

num_ent      
                                                   sum count
level_1                level_2         lvl_ent              
domain specific entity Economic sector 3             1     1
                                       4             2     1
                                       5             2     2

Let's now combine both grouped dataframes

In [220]:
# Combine matching results per entity level and domain with original number of removed entities
pd.concat([matches_within_ontology_grouped, rem_ent_ontology_grouped], axis=1)

correct_match_parent  \
                                                                     sum   
level_1                level_2              lvl_ent                        
domain specific entity Economic sector      3.0                        0   
                                            4.0                        0   
                                            5.0                        1   
                       IS technology        4.0                        0   
                                            5.0                        0   
                       IS topic             3.0                        0   
                                            4.0                       21   
                                            5.0                        2   
                       study object         4.0                        0   
                                            5.0                        1   
methodological entity  data analysis method 5.0                        1   
                                            6.0                        0   
                                            7.0                        2   
                                            8.0                        0   
                                            9.0                        0   
                       research method      5.0                        0   
                                            6.0                        5   
                                            7.0                        0   
                       sampling             4.0                        0   
                       statistical metric   3.0                        0   
                       validity             4.0                        0   
                                            5.0                        0   
theoretical entity     level of analysis    4.0                        0   
                       model                3.0                        0   
                                            4.0                        0   
                                            5.0                        0   
                                            6.0                        0   
                       research paradigm    3.0                        2   
                       theory               4.0                        1   

                                                    no_noun_phrases  \
                                                              count   
level_1                level_2              lvl_ent                   
domain specific entity Economic sector      3.0                   4   
                                            4.0                   2   
                                            5.0                   1   
                       IS technology        4.0                   4   
                                            5.0                  16   
                       IS topic             3.0                   4   
                                            4.0                  28   
                                            5.0                  11   
                       study object         4.0                   2   
                                            5.0                   3   
methodological entity  data analysis method 5.0                   1   
                                            6.0                   4   
                                            7.0                   4   
                                            8.0                   1   
                                            9.0                   8   
                       research method      5.0                   2   
                                            6.0                   8   
                                            7.0                   2   
                       sampling             4.0                   1   
                       statistical metric   3.0          